In [1]:
'''
    This script is designed to adequate the dataset to the tensorflow
    input format so that will be possible to build a classification
    model with tensorflow.

    After adequation, a model is builted using the keras API.
'''

import tensorflow as tf
from tensorflow.keras import layers, Model, metrics
import glob
from random import shuffle

In [2]:
DATASET_SIZE = 1203
BATCH_SIZE = 16
TRAIN_SIZE = 960
NUM_CLASSES = 2

filenames_list = []
labels_list = []

filenames_list = glob.glob("./lfwcrop_color/labeled_faces/*.jpg")
shuffle(filenames_list)

# last image of the non-smile list
divisor = './lfwcrop_color/labeled_faces/Jacques_Chirac_0001.jpg'

count = 1

#  Mounting the filenames and the labels list
for i in range(len(filenames_list)):
    if filenames_list[i] > divisor:
        labels_list.append(1)
    else:
        labels_list.append(0)
    
    count += 1

In [3]:
# Reads an image from a file, decodes it into a dense tensor, and resizes it
# to a fixed shape.
def _parse_function(filename, label):
    image_string = tf.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    image_resized = tf.image.resize_images(image_decoded, [64, 64])
    std_image = tf.image.per_image_standardization(image_resized)
    return std_image, label

In [4]:
# A vector of filenames.
filenames = tf.constant(filenames_list)

# 'labels[i]' is the label for the image in 'filenames[i]'.
labels = tf.constant(labels_list)
labels = tf.one_hot(tf.cast(labels, tf.int32), NUM_CLASSES)

In [5]:
def create_dataset(filenames, labels):
    # Generating tf.data.Dataset object and shuffling it
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    dataset = dataset.map(_parse_function)

    return dataset

In [6]:
def train_test_split(dataset):
    # defining batch size and 'count' number of epochs
    # taking ~ 80% for training and ~ 20% for testing
    train_data = dataset.take(TRAIN_SIZE) 
    train_data = train_data.batch(batch_size=BATCH_SIZE)
    train_data = train_data.repeat()

    test_data = dataset.skip(TRAIN_SIZE)
    test_data = test_data.batch(batch_size=BATCH_SIZE)
    test_data = test_data.repeat()

    return train_data, test_data

In [7]:
dataset = create_dataset(filenames,labels)
train_data, test_data = train_test_split(dataset)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [8]:
# --------- MODEL -------------

# Inputs
inputs = tf.keras.Input(shape=(64,64,3))

# Convolutional Layer 1
conv1 = layers.Conv2D(filters=32, kernel_size=[3,3], padding='same', activation=tf.nn.relu)(inputs)

# Pooling Layer 1
pool1 = layers.MaxPool2D(pool_size=(2,2), strides=2)(conv1)

# Convolutional Layer 2
conv2 = layers.Conv2D(filters=64, kernel_size=[3,3], activation=tf.nn.relu)(pool1)

# Pooling Layer 2
pool2 = layers.MaxPool2D(pool_size=(2,2), strides=2)(conv2)

# Convolutional Layer 2
conv3 = layers.Conv2D(filters=128, kernel_size=[3,3], activation=tf.nn.relu)(pool2)

# Pooling Layer 2
pool3 = layers.MaxPool2D(pool_size=(2,2), strides=2)(conv3)

# Flattening
pool3_flat = layers.Flatten()(pool3)

# Dense Layer 1
dense1 = layers.Dense(512, activation=tf.nn.relu)(pool3_flat)

# Dropping out with a probability of 'rate'
dropped = layers.Dropout(rate=0.5)(dense1)

# output Layer
predictions = layers.Dense(NUM_CLASSES, activation=tf.nn.softmax)(dropped)

model = Model(inputs=inputs, outputs=predictions)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
# -------- MODEL PARAMETERS ---------

# Instantiating an ADAM Optimizer
adam = tf.train.AdamOptimizer(learning_rate=0.001)

model.compile(loss=metrics.binary_crossentropy, optimizer=adam, metrics=[metrics.categorical_accuracy])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 128)         0         
__________

In [10]:
H = model.fit(
    train_data, 
    epochs=12,
    steps_per_epoch=60,
    validation_data=test_data,
    validation_steps=15
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/12
60/60 [==============================] - 9s 154ms/step - loss: 0.3464 - categorical_accuracy: 0.8500 - val_loss: 0.1640 - val_categorical_accuracy: 0.9500
Epoch 2/12
60/60 [==============================] - 8s 127ms/step - loss: 0.1824 - categorical_accuracy: 0.9344 - val_loss: 0.1287 - val_categorical_accuracy: 0.9515
Epoch 3/12
60/60 [==============================] - 8s 137ms/step - loss: 0.1195 - categorical_accuracy: 0.9604 - val_loss: 0.1220 - val_categorical_accuracy: 0.9648
Epoch 4/12
60/60 [==============================] - 8s 129ms/step - loss: 0.0730 - categorical_accuracy: 0.9823 - val_loss: 0.0884 - val_categorical_accuracy: 0.9736
Epoch 5/12
60/60 [==============================] - 8s 135ms/step - loss: 0.0516 - categorical_accuracy: 0.9865 - val_loss: 0.1129 - val_categorical_accuracy: 0.9648
Epoch 6/12
60/60 [==============================] - 8s 134ms/step - loss: 0.0496 - categorical_accuracy: 0.9885 - val_loss